In [1]:
import subprocess
import sys
import importlib

In [6]:
def check_and_install():
    """Check versions and install compatible ones if needed"""
    try:
        from transformers import __version__ as tf_version
        from datasets import __version__ as ds_version
        import torch
        
        print(f"Transformers: {tf_version}")
        print(f"Datasets: {ds_version}") 
        print(f"Torch: {torch.__version__}")
        
        # Test the problematic import
        from transformers.integrations import is_fsdp_enabled
        print("✓ All imports successful!")
        return True
        
    except ImportError as e:
        print(f"Import error: {e}")
        print("Installing compatible versions...")
        
        # Install compatible versions
        subprocess.run([
            sys.executable, "-m", "pip", "install", 
            "transformers==4.37.0", 
            "datasets==2.16.0",
            "torch==2.1.0"
        ], check=True)
        
        print("Please restart your runtime/kernel and try again.")
        return False

# Try to run the check
if not check_and_install():
    print("\n🚨 Please restart your Python kernel/runtime and run the code again.")
else:
    # Your fine-tuning code here
    print("\n✅ Ready for fine-tuning!")

Transformers: 4.57.3
Datasets: 4.4.1
Torch: 2.9.1+cpu
✓ All imports successful!

✅ Ready for fine-tuning!


In [8]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 275.1 kB/s eta 0:00:28
   -- ------------------------------------- 0.5/8.1 MB 275.1 kB/s eta 0:00:28
   -- ------------------------------------- 0.5/8.1 MB 275.1 kB/s eta 0:00:28
   -- ------------------------------------- 0.5/8.1 MB 275.1 kB/s eta 0:00:28
   -- ------------------------------------- 0.5/8.1 MB 275

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Kanan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the IMDB dataset
print("📊 Loading IMDB dataset...")
dataset = load_dataset("imdb")
print("✅ Dataset loaded successfully!")

# Inspect the dataset structure
print("\n🔍 Dataset Structure:")
print(f"Dataset type: {type(dataset)}")
print(f"Dataset keys: {list(dataset.keys())}")

print("\n📋 Dataset Features:")
print(dataset['train'].features)

# Let's examine the training and test datasets
print(f"\n📊 Dataset splits:")
for split_name, split_data in dataset.items():
    print(f"{split_name}: {len(split_data)} examples")

# Sample some data to understand the format
print("\n👀 Sample data from training set:")
for i in range(3):
    print(f"\nSample {i+1}:")
    print(f"Text: {dataset['train'][i]['text'][:200]}...")  # First 200 chars
    print(f"Label: {dataset['train'][i]['label']} (0 = negative, 1 = positive)")

# Analyze data distribution
print("\n📈 Data Distribution Analysis:")

# Check label distribution in training set
train_labels = dataset['train']['label']
test_labels = dataset['test']['label']

print(f"\nTraining set label distribution:")
unique, counts = np.unique(train_labels, return_counts=True)
for label, count in zip(unique, counts):
    sentiment = "negative" if label == 0 else "positive"
    percentage = (count / len(train_labels)) * 100
    print(f"  {sentiment} ({label}): {count} examples ({percentage:.1f}%)")

print(f"\nTest set label distribution:")
unique, counts = np.unique(test_labels, return_counts=True)
for label, count in zip(unique, counts):
    sentiment = "negative" if label == 0 else "positive"
    percentage = (count / len(test_labels)) * 100
    print(f"  {sentiment} ({label}): {count} examples ({percentage:.1f}%)")

# Check text length statistics
print(f"\n📏 Text Length Analysis:")
train_text_lengths = [len(text.split()) for text in dataset['train']['text']]
test_text_lengths = [len(text.split()) for text in dataset['test']['text']]

print(f"Training set - Avg words: {np.mean(train_text_lengths):.1f}, "
      f"Max: {np.max(train_text_lengths)}, Min: {np.min(train_text_lengths)}")
print(f"Test set - Avg words: {np.mean(test_text_lengths):.1f}, "
      f"Max: {np.max(test_text_lengths)}, Min: {np.min(test_text_lengths)}")

# Visualize the distribution (optional)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(train_text_lengths, bins=50, alpha=0.7, color='blue', label='Train')
plt.hist(test_text_lengths, bins=50, alpha=0.7, color='red', label='Test')
plt.xlabel('Text Length (words)')
plt.ylabel('Frequency')
plt.title('Text Length Distribution')
plt.legend()
plt.yscale('log')  # Log scale for better visualization

plt.subplot(1, 2, 2)
labels = ['Negative', 'Positive']
train_counts = [np.sum(train_labels == 0), np.sum(train_labels == 1)]
test_counts = [np.sum(test_labels == 0), np.sum(test_labels == 1)]

x = np.arange(len(labels))
width = 0.35

plt.bar(x - width/2, train_counts, width, label='Train', alpha=0.7)
plt.bar(x + width/2, test_counts, width, label='Test', alpha=0.7)
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Label Distribution')
plt.xticks(x, labels)
plt.legend()

plt.tight_layout()
plt.show()

# Summary of findings
print("\n" + "="*50)
print("📋 DATASET SUMMARY")
print("="*50)
print("✅ Inputs: Movie review texts (strings)")
print("✅ Outputs: Binary sentiment labels (0 = negative, 1 = positive)")
print("✅ Labeling: Human-annotated sentiment")
print(f"✅ Training set: {len(dataset['train'])} balanced examples")
print(f"✅ Test set: {len(dataset['test'])} balanced examples")
print("✅ Data distribution: Perfectly balanced between positive/negative")
print("✅ Text length: Variable, typically a few hundred words")
print("✅ Further manipulation: May need tokenization and padding for model input")
print("="*50)

ModuleNotFoundError: No module named 'matplotlib'